In [54]:
#import sys
#import subprocess

#subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "numpy", "statsmodels", "matplotlib", "scipy", "openpyxl"])

In [55]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit
import matplotlib.pyplot as plt
import scipy.stats as stats
import openpyxl as xl


# Data Preparation

Laod the rates.xlsx File and perform a stadard date fromatting
create a new variable that computes the spread betewwen the 10y DGS10 and the 3 Month DGS3MO yealds by substracting the 3 month from the 10 Year
Aggregate the daily Data to the montly frequencey, keeping the last daily observation for each month

In [56]:
rates = pd.read_excel('rates.xlsx', engine='openpyxl')

# Clean up and sort the date column
rates['observation_date'] = pd.to_datetime(rates['observation_date'])
rates = rates.sort_values('observation_date').reset_index(drop=True)

# Compute term spread between 10Y and 3M yields
rates['term_spread_10y_3m'] = rates['DGS10'] - rates['DGS3MO']

# Aggregate to month end, keeping the last observation of each month
rates_monthly = (
    rates.set_index('observation_date')
         .resample('ME')
         .last()
         .reset_index()
)

rates_monthly.head()

,observation_date,fed_funds_futures,credit_spread,5y5y,DGS3MO,DGS10,term_spread_10y_3m
0,2010-01-31,99.8875,1.83,2.75,0.08,3.63,3.55
1,2010-02-28,99.8725,1.84,2.43,0.13,3.61,3.48
2,2010-03-31,99.8325,1.61,2.65,0.16,3.84,3.68
3,2010-04-30,99.8000,1.55,2.79,0.16,3.69,3.53
4,2010-05-31,99.7950,2.02,2.29,0.16,3.31,3.15


Q1 Waht is the minimum FED Funds Futures price in your dataset

In [57]:
min_ff_price = rates['fed_funds_futures'].min()
min_ff_date = rates.loc[rates['fed_funds_futures'].idxmin(), 'observation_date']
print(f"Minimum Fed Funds Futures price: {min_ff_price:.3f} on {min_ff_date.date()}")

Minimum Fed Funds Futures price: 94.662 on 2023-09-05


Maximum credit spread in YOur Datasead

In [58]:
max_credit_spread = rates['credit_spread'].max()
max_credit_date = rates.loc[rates['credit_spread'].idxmax(), 'observation_date']
print(f"Maximum credit spread: {max_credit_spread:.2f} on {max_credit_date.date()}")

Maximum credit spread: 4.01 on 2020-03-23


Laod the stocks.xlsx File and perform a stadard date fromatting sort observation from oldest to newest
Aggregate the daily colsing price (labled "close") to the montly frequencey, keeping the last  observation for each month


In [59]:
from pathlib import Path
stock_file = Path('stocks.xlsx') if Path('stocks.xlsx').exists() else Path('stock.xlsx')
stocks = pd.read_excel(stock_file, engine='openpyxl')

# Standardize column names and ensure date handling
stocks = stocks.rename(columns=str.lower)
stocks['observation_date'] = pd.to_datetime(stocks['observation_date'])
stocks = stocks.sort_values('observation_date').reset_index(drop=True)

# Aggregate daily close to month-end values
stocks_monthly = (
    stocks[['observation_date', 'close']]
         .set_index('observation_date')
         .resample('ME')
         .last()
         .reset_index()
)

stocks_monthly.head()

,observation_date,close
0,2010-01-31,27.125
1,2010-02-28,26.360
2,2010-03-31,27.500
3,2010-04-30,26.725
4,2010-05-31,25.700


Average stock price

In [60]:
avg_monthly_close = stocks_monthly['close'].mean()
print(f"Average monthly close: {avg_monthly_close:.2f}")

Average monthly close: 47.55


Load mkt.xlsx standard formatting
devide both market excess retrun Mkt-RF and RF by 100 both in new variables
Merge stock dataframe with rates Data Frame Moth als combinierende Variable
Use the monthly stock price to compute a simple montly stock returns
Compute montly excess stock retruns by RF minus der ,ontly retrun varibale
Für die Fed Funds Futures credit spread 5y-5y und die 10y-3M yield spread compute the montly cahnge by substracting the the previos month value from the current mont value
Drop all missing Values

In [1]:
from pandas.tseries.offsets import MonthEnd

# Load market excess returns and risk-free rate
mkt = pd.read_excel('mkt.xlsx', engine='openpyxl')
mkt.columns = (
    mkt.columns.str.strip()
               .str.lower()
               .str.replace('-', '_')
)
mkt['month'] = pd.to_datetime(mkt['month'].astype(str), format='%Y%m')
mkt[['mkt_rf', 'rf']] = mkt[['mkt_rf', 'rf']].apply(lambda s: pd.to_numeric(s, errors='coerce'))
mkt['mkt_rf_dec'] = mkt['mkt_rf'] / 100
mkt['rf_dec'] = mkt['rf'] / 100
mkt['observation_date'] = mkt['month'] + MonthEnd(0)

# Load US recession indicator and standardize formatting
usrec = pd.read_excel('USREC-2.xlsx', engine='openpyxl')
usrec.columns = usrec.columns.str.strip().str.lower()
date_col = 'observation_date' if 'observation_date' in usrec.columns else usrec.columns[0]
usrec = usrec.rename(columns={date_col: 'observation_date'})
indicator_cols = [col for col in usrec.columns if col != 'observation_date']
indicator_col = indicator_cols[0] if indicator_cols else None
if indicator_col is None:
    raise ValueError('US recession file must contain an indicator column.')
usrec = usrec[['observation_date', indicator_col]].copy()
usrec['observation_date'] = pd.to_datetime(usrec['observation_date'])
usrec = usrec.sort_values('observation_date').reset_index(drop=True)
usrec_monthly = (
    usrec.set_index('observation_date')
         .resample('ME')
         .last()
         .reset_index()
         .rename(columns={indicator_col: 'us_recession'})
)
usrec_monthly['us_recession'] = usrec_monthly['us_recession'].ffill().bfill().astype(int)

# Prepare stock and rates datasets on the same month-end key
stocks_monthly = stocks_monthly.sort_values('observation_date').copy()
rates_monthly = rates_monthly.sort_values('observation_date').copy()

combined_monthly = (
    stocks_monthly
        .merge(rates_monthly, on='observation_date', how='left', suffixes=('', '_rates'))
        .merge(mkt[['observation_date', 'mkt_rf_dec', 'rf_dec']], on='observation_date', how='left')
        #.merge(usrec_monthly[['observation_date', 'us_recession']], on='observation_date', how='left')
        .sort_values('observation_date')
)

combined_monthly['stock_return'] = combined_monthly['close'].pct_change()
combined_monthly['stock_excess_return'] = combined_monthly['stock_return'] - combined_monthly['rf_dec']
for col in ['fed_funds_futures', 'credit_spread', '5y5y', 'term_spread_10y_3m']:
    combined_monthly[f'{col}_chg'] = combined_monthly[col].diff()

combined_monthly['us_recession'] = combined_monthly['us_recession'].ffill().bfill().astype(int)
combined_monthly = combined_monthly.dropna().reset_index(drop=True)
combined_monthly.head()

NameError: name 'pd' is not defined

Wie viele observations habe ich

In [62]:
num_observations = len(combined_monthly)
print(f'Number of observations after cleaning: {num_observations}')

Number of observations after cleaning: 187


Sharp ratio of my stocks

In [63]:
# Compute Sharpe ratio based on monthly excess returns
sharpe_ratio = combined_monthly['stock_excess_return'].mean() / combined_monthly['stock_excess_return'].std()
annualized_sharpe = sharpe_ratio * np.sqrt(12)

print(f'Monthly Sharpe ratio: {sharpe_ratio:.3f}')
print(f'Annualized Sharpe ratio: {annualized_sharpe:.3f}')

combined_monthly.head()


Monthly Sharpe ratio: 0.112
Annualized Sharpe ratio: 0.388


,observation_date,close,fed_funds_futures,credit_spread,5y5y,DGS3MO,DGS10,term_spread_10y_3m,mkt_rf_dec,rf_dec,us_recession,stock_return,stock_excess_return,fed_funds_futures_chg,credit_spread_chg,5y5y_chg,term_spread_10y_3m_chg
0,2010-02-28,26.360,99.8725,1.84,2.43,0.13,3.61,3.48,0.0339,0.0000,0,-0.028203,-0.028203,-0.0150,0.01,-0.32,-0.07
1,2010-03-31,27.500,99.8325,1.61,2.65,0.16,3.84,3.68,0.0630,0.0001,0,0.043247,0.043147,-0.0400,-0.23,0.22,0.20
2,2010-04-30,26.725,99.8000,1.55,2.79,0.16,3.69,3.53,0.0199,0.0001,0,-0.028182,-0.028282,-0.0325,-0.06,0.14,-0.15
3,2010-05-31,25.700,99.7950,2.02,2.29,0.16,3.31,3.15,-0.0790,0.0001,0,-0.038354,-0.038454,-0.0050,0.47,-0.50,-0.38
4,2010-06-30,25.060,99.8225,2.09,2.10,0.18,2.97,2.79,-0.0556,0.0001,0,-0.024903,-0.025003,0.0275,0.07,-0.19,-0.36


Ols Regression

In [67]:
predictors = [
    "fed_funds_futures_chg",
    "credit_spread_chg",
    "5y5y_chg",
    "term_spread_10y_3m_chg",
]
max_lag = 12

X_current = combined_monthly[predictors]
lagged_blocks = [X_current]
for lag in range(1, max_lag + 1):
    lagged_block = X_current.shift(lag)
    lagged_block.columns = [f"{col}_lag{lag}" for col in predictors]
    lagged_blocks.append(lagged_block)

X_full = pd.concat(lagged_blocks, axis=1)
regression_df = pd.concat([combined_monthly[["mkt_rf_dec"]], X_full], axis=1).dropna()

Y = regression_df["mkt_rf_dec"]
X = regression_df.drop(columns="mkt_rf_dec")

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             mkt_rf_dec   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     4.944
Date:                Thu, 23 Oct 2025   Prob (F-statistic):           2.12e-13
Time:                        20:13:13   Log-Likelihood:                 403.44
No. Observations:                 175   AIC:                            -700.9
Df Residuals:                     122   BIC:                            -533.2
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            0.0091      0.002      3.712      0.000       0.004       0.014
fed_funds_futures_chg            0.0996      0.031      3.201      0.002       0.038       0.161
credit_spread_chg               -0.1456      0.019     -7.857      0.000      -0.182      -0.109
5y5y_chg                         0.0688      0.024      2.923      0.004       0.022       0.115
term_spread_10y_3m_chg          -0.0418      0.012     -3.459      0.001      -0.066      -0.018
fed_funds_futures_chg_lag1      -0.0499      0.040     -1.257      0.211      -0.129       0.029
credit_spread_chg_lag1          -0.0128      0.019     -0.673      0.502      -0.050       0.025
5y5y_chg_lag1                   -0.0148      0.025     -0.603      0.548      -0.063       0.034
term_spread_10y_3m_chg_lag1      0.0041      0.012      0.335      0.739      -0.020       0.029
fed_funds_futures_chg_lag2       0.0285      0.040      0.716      0.475      -0.050       0.107
credit_spread_chg_lag2          -0.0207      0.019     -1.080      0.282      -0.059       0.017
5y5y_chg_lag2                   -0.0541      0.025     -2.172      0.032      -0.103      -0.005
term_spread_10y_3m_chg_lag2      0.0156      0.013      1.157      0.250      -0.011       0.042
fed_funds_futures_chg_lag3      -0.0566      0.040     -1.427      0.156      -0.135       0.022
credit_spread_chg_lag3           0.0016      0.020      0.080      0.937      -0.038       0.041
5y5y_chg_lag3                    0.0158      0.025      0.623      0.534      -0.034       0.066
term_spread_10y_3m_chg_lag3     -0.0257      0.013     -1.912      0.058      -0.052       0.001
fed_funds_futures_chg_lag4       0.0191      0.039      0.484      0.629      -0.059       0.097
credit_spread_chg_lag4          -0.0184      0.020     -0.928      0.355      -0.058       0.021
5y5y_chg_lag4                    0.0118      0.024      0.490      0.625      -0.036       0.059
term_spread_10y_3m_chg_lag4   3.552e-05      0.013      0.003      0.998      -0.027       0.027
fed_funds_futures_chg_lag5      -0.0652      0.040     -1.638      0.104      -0.144       0.014
credit_spread_chg_lag5           0.0271      0.019      1.427      0.156      -0.011       0.065
5y5y_chg_lag5                    0.0114      0.023      0.501      0.617      -0.034       0.056
term_spread_10y_3m_chg_lag5      0.0146      0.013      1.133      0.259      -0.011       0.040
fed_funds_futures_chg_lag6       0.0273      0.039      0.700      0.485      -0.050       0.105
credit_spread_chg_lag6          -0.0181      0.019     -0.939      0.350      -0.056       0.020
5y5y_chg_lag6                   -0.0343      0.023     -1.507      0.134      -0.079       0.011
term_spread_10y_3m_chg_lag6     -0.0048      0.013     -0.384      0.702    

Load USREC-2, stadrd formating
combine dataframe combine monthly leading nach month
Ols regression x = monthly stock return
y = US recession indicator
12 lags
Frage  waht is the dofference of your stocks return on average per montah during the recession comparet to the non recession peroides in percentage points

Difference in average stock returns during recession vs. non-recession periods

In [65]:
recession_mask = combined_monthly['us_recession'] == 1
avg_return_recession = combined_monthly.loc[recession_mask, 'stock_return'].mean()
avg_return_non_recession = combined_monthly.loc[~recession_mask, 'stock_return'].mean()

diff_percentage_points = (avg_return_recession - avg_return_non_recession) * 100

print(f'Average monthly stock return during recession: {avg_return_recession * 100:.2f}%')
print(f'Average monthly stock return outside recession: {avg_return_non_recession * 100:.2f}%')
print(f'Difference (recession - non-recession): {diff_percentage_points:.2f} percentage points')

Average monthly stock return during recession: -6.78%
Average monthly stock return outside recession: 0.68%
Difference (recession - non-recession): -7.46 percentage points


OLS regression: US recession indicator on current and lagged stock returns (12 lags)

In [66]:
max_lag = 12
return_series = combined_monthly[['stock_return']]
lagged_returns = [return_series]
for lag in range(1, max_lag + 1):
    lagged = return_series.shift(lag).rename(columns={'stock_return': f'stock_return_lag{lag}'})
    lagged_returns.append(lagged)

X_returns = pd.concat(lagged_returns, axis=1)
recession_reg_df = pd.concat([combined_monthly[['us_recession']], X_returns], axis=1).dropna()

Y = recession_reg_df['us_recession']
X = recession_reg_df.drop(columns='us_recession')

X = sm.add_constant(X)
recession_model = sm.OLS(Y, X).fit()
recession_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           us_recession   R-squared:                       0.212
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     3.335
Date:                Thu, 23 Oct 2025   Prob (F-statistic):           0.000156
Time:                        20:07:28   Log-Likelihood:                 164.83
No. Observations:                 175   AIC:                            -301.7
Df Residuals:                     161   BIC:                            -257.3
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0173      0.010      1.733      0.085      -0.002       0.037
stock_return          -0.6285      0.180     -3.487      0.001      -0.984      -0.273
stock_return_lag1     -0.9298      0.183     -5.080      0.000      -1.291      -0.568
stock_return_lag2     -0.3308      0.188     -1.764      0.080      -0.701       0.040
stock_return_lag3      0.0612      0.188      0.326      0.745      -0.310       0.432
stock_return_lag4     -0.0639      0.186     -0.344      0.731      -0.431       0.303
stock_return_lag5     -0.1454      0.186     -0.781      0.436      -0.513       0.222
stock_return_lag6     -0.1434      0.186     -0.773      0.441      -0.510       0.223
stock_return_lag7      0.0127      0.185      0.069      0.945      -0.352       0.378
stock_return_lag8      0.2426      0.184      1.321      0.188      -0.120       0.605
stock_return_lag9      0.2493      0.186      1.341      0.182      -0.118       0.616
stock_return_lag10     0.1280      0.186      0.688      0.492      -0.239       0.495
stock_return_lag11     0.1330      0.185      0.718      0.474      -0.233       0.499
stock_return_lag12     0.2090      0.182      1.149      0.252      -0.150       0.568
==============================================================================
Omnibus:                      255.291   Durbin-Watson:                   1.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19578.161
Skew:                           6.376   Prob(JB):                         0.00
Kurtosis:                      53.223   Cond. No.                         41.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""